In [8]:
import requests
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import time
import datetime as dt

print('Imports completed!')

Imports completed!


# TEAMS

In [9]:
url = "https://api-nba-v1.p.rapidapi.com/teams"
headers = {
	"x-rapidapi-key": "dda6cf782dmshebe3119a485b548p154863jsn7c9c31f1e7a7",
	"x-rapidapi-host": "api-nba-v1.p.rapidapi.com"
}

# Extract from url and header parameters
def extract_data(api_url, headers):
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"API request failed with status code: {response.status_code}")

# Clean the data
def transform_data(raw_data):
    # ID's to not be included
    exclude_ids = {37, 34, 39, 102, 103, 42, 35, 36, 49}

    # Gather Data for Teams
    teams = [
        {
            'team_id': row['id'],
            'team_code': row['code'],
            'team_name': row['name']
        }
        for row in raw_data['response']
        if 'leagues' in row and 'standard' in row['leagues']
        and row['leagues']['standard']['division'] is not None
        and row['id'] not in exclude_ids
    ]
    
    teams_df = pd.DataFrame(teams)
    return teams_df

# Load data into a table
def load_data(df, table_name, connection_string):
    engine = create_engine(connection_string)
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f'Data has been loaded and added to the {table_name} database')

In [10]:
# Run data pipeline
def run_pipeline():
    raw_data = extract_data(url, headers)
    clean_data = transform_data(raw_data)
    connection_string = 'sqlite:///teams_data.db'
    load_data(clean_data, 'teams', connection_string)
    
run_pipeline()

Data has been loaded and added to the teams database


# GAME SCORES

In [11]:

import pandas as pd
from sqlalchemy import create_engine

def read_data(connection_string):
    engine = create_engine(connection_string)
    query = "SELECT * FROM teams"
    df = pd.read_sql(query, con=engine)
    return df

# Read the data back from the database
connection_string = 'sqlite:///teams_data.db'
teams_data = read_data(connection_string)

# Export DataFrame to Excel
teams_data.to_excel('./data/team_data.xlsx', index=False)

# Display the data
teams_data

,team_id,team_code,team_name
0,1,ATL,Atlanta Hawks
1,2,BOS,Boston Celtics
2,4,BKN,Brooklyn Nets
3,5,CHA,Charlotte Hornets
4,6,CHI,Chicago Bulls
5,7,CLE,Cleveland Cavaliers
6,8,DAL,Dallas Mavericks
7,9,DEN,Denver Nuggets
8,10,DET,Detroit Pistons
9,11,GSW,Golden State Warriors


In [12]:
from itertools import combinations 
import time

url = "https://api-nba-v1.p.rapidapi.com/games"
headers = {
	"x-rapidapi-key": "dda6cf782dmshebe3119a485b548p154863jsn7c9c31f1e7a7",
	"x-rapidapi-host": "api-nba-v1.p.rapidapi.com"
}

team_ids = teams_data['team_id'].values

def game_matches(team_ids, url, headers):
    unique_matchups = list(combinations(team_ids, 2))

    response_arr = []
    for game in unique_matchups:
        querystring = {"h2h": f"{game[0]}-{game[1]}"}
        print(f"Gathering data for matchup {game[0]}-{game[1]}")

        response = requests.get(url, headers=headers, params=querystring)

        if response.status_code == 200:
            game_data = response.json()
            response_arr.append(game_data)
        else:
            print(f"Failed to gather data for {game[0]}-{game[1]}. Status code: {response.status_code}")

        time.sleep(10)

    print("Data Collection Completed!")
    return response_arr

results = game_matches(team_ids, url, headers)

Gathering data for matchup 1-2
Gathering data for matchup 1-4
Gathering data for matchup 1-5
Gathering data for matchup 1-6
Gathering data for matchup 1-7
Gathering data for matchup 1-8
Gathering data for matchup 1-9
Gathering data for matchup 1-10
Gathering data for matchup 1-11
Gathering data for matchup 1-14
Gathering data for matchup 1-15
Gathering data for matchup 1-16
Gathering data for matchup 1-17
Gathering data for matchup 1-19
Gathering data for matchup 1-20
Gathering data for matchup 1-21
Gathering data for matchup 1-22
Gathering data for matchup 1-23
Gathering data for matchup 1-24
Gathering data for matchup 1-25
Gathering data for matchup 1-26
Gathering data for matchup 1-27
Gathering data for matchup 1-28
Gathering data for matchup 1-29
Gathering data for matchup 1-30
Gathering data for matchup 1-31
Gathering data for matchup 1-38
Gathering data for matchup 1-40
Gathering data for matchup 1-41
Gathering data for matchup 2-4
Gathering data for matchup 2-5
Gathering data fo

In [13]:
game_scores = []
for info in results:
    for game in info['response']:
        # Extract id for each game
        game_ids = game.get('id', 'N/A')
        
        # Extract date of the game
        date = game.get('date', {}).get('start','N/A')
        
        # Extract teams info
        visitor = game.get('teams', {}).get('visitors', {}).get('name', 'Unknown')
        home = game.get('teams', {}).get('home', {}).get('name', 'Unknown')
     
        visitor_code = game.get('teams', {}).get('visitors', {}).get('code', 'N/A')
        home_code = game.get('teams', {}).get('home', {}).get('code', 'N/A') 
        
        # Initialize lists to store total scores
        visitor_total_scores = []
        home_total_scores = []
        
        # Extract scores by quarter
        visitor_scores = game.get('scores', {}).get('visitors', {}).get('linescore', [])
        home_scores = game.get('scores', {}).get('home', {}).get('linescore', [])
        
        visitor_points = game.get('scores', {}).get('visitors', {}).get('points', '0')
        home_points = game.get('scores', {}).get('home', {}).get('points', '0')
        
        if visitor_scores and home_scores and visitor_points is not None and home_points is not None:
            visitor_total_scores.append(visitor_points)
            home_total_scores.append(home_points)
        
        # Display data if scores and points exist
        if visitor_scores and home_scores and visitor_total_scores and home_total_scores:
            print(f"Date: {date}")
            print(f"Visitor Team: {visitor} ({visitor_code}) - Scores: {visitor_scores} - Total Points: {visitor_total_scores[0]}")
            print(f"Home Team: {home} ({home_code}) - Scores: {home_scores} - Total Points: {home_total_scores[0]}")
            print('-------------------------------------------')
            
        game_scores.append({
            'game_ids': game_ids,
            'date': date,
            'visitor': visitor,
            'home': home,
            'visitor_points': visitor_points,
            'home_points': home_points
        })
        
    game_score_df = pd.DataFrame(game_scores)

Date: 2015-11-25T01:00:00.000Z
Visitor Team: Boston Celtics (BOS) - Scores: ['28', '20', '25', '24'] - Total Points: 97
Home Team: Atlanta Hawks (ATL) - Scores: ['33', '24', '24', '40'] - Total Points: 121
-------------------------------------------
Date: 2016-04-09T23:30:00.000Z
Visitor Team: Boston Celtics (BOS) - Scores: ['32', '39', '20', '16'] - Total Points: 107
Home Team: Atlanta Hawks (ATL) - Scores: ['34', '33', '26', '25'] - Total Points: 118
-------------------------------------------
Date: 2016-04-16T23:00:00.000Z
Visitor Team: Boston Celtics (BOS) - Scores: ['19', '15', '31', '36'] - Total Points: 101
Home Team: Atlanta Hawks (ATL) - Scores: ['30', '21', '21', '30'] - Total Points: 102
-------------------------------------------
Date: 2016-04-19T23:00:00.000Z
Visitor Team: Boston Celtics (BOS) - Scores: ['7', '21', '18', '26'] - Total Points: 72
Home Team: Atlanta Hawks (ATL) - Scores: ['24', '19', '18', '28'] - Total Points: 89
-------------------------------------------


In [14]:
game_score_df.to_excel('./data/apiGameScores.xlsx', index=False)
game_score_df

,game_ids,date,visitor,home,visitor_points,home_points
0,319,2015-11-25T01:00:00.000Z,Boston Celtics,Atlanta Hawks,97.0,121.0
1,1300,2016-04-09T23:30:00.000Z,Boston Celtics,Atlanta Hawks,107.0,118.0
2,1344,2016-04-16T23:00:00.000Z,Boston Celtics,Atlanta Hawks,101.0,102.0
3,1353,2016-04-19T23:00:00.000Z,Boston Celtics,Atlanta Hawks,72.0,89.0
4,1376,2016-04-27T00:30:00.000Z,Boston Celtics,Atlanta Hawks,83.0,110.0
...,...,...,...,...,...,...
14237,9048,2021-04-13T01:00:00.000Z,Washington Wizards,Utah Jazz,125.0,121.0
14238,10012,2021-12-19T02:00:00.000Z,Washington Wizards,Utah Jazz,109.0,103.0
14239,11527,2022-12-23T02:00:00.000Z,Washington Wizards,Utah Jazz,112.0,120.0
14240,13430,2024-03-05T02:00:00.000Z,Washington Wizards,Utah Jazz,115.0,127.0


In [15]:
game_score_df.loc[(game_score_df['visitor'] == 'Boston Celtics') & (game_score_df['home'] == 'Atlanta Hawks')]

,game_ids,date,visitor,home,visitor_points,home_points
0,319,2015-11-25T01:00:00.000Z,Boston Celtics,Atlanta Hawks,97.0,121.0
1,1300,2016-04-09T23:30:00.000Z,Boston Celtics,Atlanta Hawks,107.0,118.0
2,1344,2016-04-16T23:00:00.000Z,Boston Celtics,Atlanta Hawks,101.0,102.0
3,1353,2016-04-19T23:00:00.000Z,Boston Celtics,Atlanta Hawks,72.0,89.0
4,1376,2016-04-27T00:30:00.000Z,Boston Celtics,Atlanta Hawks,83.0,110.0
5,2125,2017-01-14T01:00:00.000Z,Boston Celtics,Atlanta Hawks,103.0,101.0
6,2709,2017-04-07T00:00:00.000Z,Boston Celtics,Atlanta Hawks,116.0,123.0
7,3065,2017-11-07T00:30:00.000Z,Boston Celtics,Atlanta Hawks,110.0,107.0
8,3149,2017-11-19T00:30:00.000Z,Boston Celtics,Atlanta Hawks,110.0,99.0
9,4654,2018-11-24T00:30:00.000Z,Boston Celtics,Atlanta Hawks,114.0,96.0


# Game Statistics Data Extraction

In [34]:
# Read in game score data
game_df = pd.read_excel('./data/apiGameScores.xlsx')

In [35]:
game_df.isnull().sum()

game_ids             0
date                 0
visitor              0
home                 0
visitor_points    1290
home_points       1290
dtype: int64

In [36]:
# Drop rows with null values
game_df = game_df.dropna()

# Shape of the data frame
game_df.shape

(12952, 6)

In [82]:
# Create a Data Frame within specific time frame date
recent_games = game_df.loc[game_df['date'] >= '2024-01-01']
recent_games.reset_index(drop=True)

,game_ids,date,visitor,home,visitor_points,home_points
0,13582,2024-03-25T23:30:00.000Z,Boston Celtics,Atlanta Hawks,118.0,120.0
1,13608,2024-03-28T23:30:00.000Z,Boston Celtics,Atlanta Hawks,122.0,123.0
2,14211,2024-11-05T00:45:00.000Z,Boston Celtics,Atlanta Hawks,123.0,93.0
3,13277,2024-02-08T00:30:00.000Z,Atlanta Hawks,Boston Celtics,117.0,125.0
4,14271,2024-11-13T00:00:00.000Z,Atlanta Hawks,Boston Celtics,117.0,116.0
...,...,...,...,...,...,...
1172,14050,2024-10-06T23:30:00.000Z,Washington Wizards,Toronto Raptors,98.0,125.0
1173,13570,2024-03-24T00:00:00.000Z,Toronto Raptors,Washington Wizards,109.0,112.0
1174,14072,2024-10-11T23:00:00.000Z,Toronto Raptors,Washington Wizards,95.0,113.0
1175,13176,2024-01-26T00:00:00.000Z,Utah Jazz,Washington Wizards,123.0,108.0


In [83]:
# Grab game ids to be used with api call
game_ids = recent_games['game_ids'].values
print(f"Grabbing data for {len(recent_games)} games")

Grabbing data for 1177 games


In [84]:
game_ids

array([13582, 13608, 14211, ..., 14072, 13176, 13430], dtype=int64)

In [ ]:
url = "https://api-nba-v1.p.rapidapi.com/games/statistics"
headers = {
    "x-rapidapi-key": "dda6cf782dmshebe3119a485b548p154863jsn7c9c31f1e7a7",
    "x-rapidapi-host": "api-nba-v1.p.rapidapi.com"
}

statistics = []
processed_games = 0 

for game_id in game_ids:
    querystring = {"id": game_id}
    response = requests.get(url, headers=headers, params=querystring, timeout=10)
    
     # Check if the API request was successful
    if response.status_code == 200:
        game_statistics = response.json()
        
        if len(game_statistics.get('response', [])) > 1:
            # Extract game id
            id = game_statistics['parameters']['id']

            # # Extract visitor and home team names
            # visitor = game_statistics['response'][0].get('team', {}).get('name', 'Unknown')
            # home = game_statistics['response'][1].get('team', {}).get('name', 'Unknown')

            # Extract statistics for visitor team
            visitor_statics = game_statistics['response'][0]['statistics'][0]
            visitor_stats = {f"visitor_{key}":value for key, value in visitor_statics.items()}
            visitor_stats['game_id'] = id
            visitor_stats['visitor_team'] = visitor
            
            # Extract statistics for home team
            home_statics = game_statistics['response'][1]['statistics'][0]
            home_stats = {f"home_{key}":value for key, value in home_statics.items()}
            home_stats['game_id'] = id
            home_stats['home_team'] = home

            # Append both visitor and home stats
            statistics.append({**visitor_stats, **home_stats})
            processed_games += 1
            
            print(f"Grabbing game statistics from game id: {game_id}")
            print(f"Total Games Processed: {processed_games}")
   
        else:
            print(f"No statistics found for Game ID: {game_id}")
            
    else:
        print(f"Failed to fetch data for Game ID: {game_id}. Status Code: {response.status_code}")
     
    # Sleep function
    time.sleep(10)
     
# Create the Data Frame
game_statistics_df = pd.DataFrame(statistics)

Grabbing game statistics from game id: 13582
Total Games Processed: 1
Grabbing game statistics from game id: 13608
Total Games Processed: 2
Grabbing game statistics from game id: 14211
Total Games Processed: 3
Grabbing game statistics from game id: 13277
Total Games Processed: 4
Grabbing game statistics from game id: 14271
Total Games Processed: 5
Grabbing game statistics from game id: 14118
Total Games Processed: 6
Grabbing game statistics from game id: 13399
Total Games Processed: 7
Grabbing game statistics from game id: 13414
Total Games Processed: 8
Grabbing game statistics from game id: 13569
Total Games Processed: 9
Grabbing game statistics from game id: 13707
Total Games Processed: 10
Grabbing game statistics from game id: 14133
Total Games Processed: 11
Grabbing game statistics from game id: 13325
Total Games Processed: 12
Grabbing game statistics from game id: 13311
Total Games Processed: 13
Grabbing game statistics from game id: 14252
Total Games Processed: 14
Grabbing game s

In [ ]:
# Export as csv file
game_statistics_df.to_csv('./data/gameStatistics.csv', index=False)
game_statistics_df

,visitor_fastBreakPoints,visitor_pointsInPaint,visitor_biggestLead,visitor_secondChancePoints,visitor_pointsOffTurnovers,visitor_longestRun,visitor_points,visitor_fgm,visitor_fga,visitor_fgp,...,home_defReb,home_totReb,home_assists,home_pFouls,home_steals,home_turnovers,home_blocks,home_plusMinus,home_min,home_team
0,6.0,38.0,3.0,9.0,13.0,11.0,92.0,30.0,86.0,34.9,...,34.0,43.0,19.0,21.0,10.0,11.0,11.0,16,240:00,Utah Jazz
1,NaN,NaN,NaN,NaN,NaN,NaN,101.0,42.0,101.0,41.6,...,43.0,49.0,29.0,20.0,4.0,15.0,8.0,25,240:00,Utah Jazz
2,NaN,NaN,NaN,NaN,NaN,NaN,130.0,51.0,91.0,56.0,...,23.0,29.0,31.0,17.0,11.0,12.0,0.0,-8,240:00,Utah Jazz
3,NaN,NaN,NaN,NaN,NaN,NaN,121.0,43.0,98.0,43.9,...,40.0,49.0,25.0,23.0,4.0,16.0,5.0,8,240:00,Utah Jazz
4,NaN,NaN,NaN,NaN,NaN,NaN,120.0,44.0,94.0,46.8,...,36.0,47.0,24.0,16.0,7.0,7.0,10.0,8,240:00,Utah Jazz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3896,NaN,NaN,NaN,NaN,NaN,NaN,113.0,35.0,76.0,46.1,...,26.0,45.0,20.0,32.0,12.0,23.0,3.0,-18,240:00,Utah Jazz
3897,NaN,NaN,NaN,NaN,NaN,NaN,121.0,44.0,88.0,50.0,...,35.0,42.0,33.0,17.0,4.0,12.0,3.0,-9,240:00,Utah Jazz
3898,NaN,NaN,NaN,NaN,NaN,NaN,108.0,43.0,88.0,48.9,...,39.0,52.0,34.0,19.0,6.0,16.0,2.0,15,240:00,Utah Jazz
3899,NaN,NaN,NaN,NaN,NaN,NaN,120.0,42.0,86.0,48.8,...,34.0,37.0,20.0,15.0,5.0,11.0,2.0,-8,240:00,Utah Jazz
